In [1]:
import os
from openai import OpenAI

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [2]:
def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of a location, the user should supply a json string with many detailed info, including location, date, temperture, etc, please help to parse them",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA or 石景山，北京 etc.",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

In [4]:
messages = [{"role": "user", "content": "What is the weather of 北京三里屯? Please show in markdown format and in Chinese"}]
message = send_messages(messages)

In [5]:
print(messages)
print('\n')
print(message)

[{'role': 'user', 'content': 'What is the weather of 北京三里屯? Please show in markdown format and in Chinese'}]


ChatCompletionMessage(content='我来帮您查询北京三里屯的天气情况。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_00_ta83vX1tilstuHNeeLGqf5x1', function=Function(arguments='{"location": "北京三里屯"}', name='get_weather'), type='function', index=0)])


In [6]:
print(f"User>\t {messages[0]['content']}")

User>	 What is the weather of 北京三里屯? Please show in markdown format and in Chinese


In [7]:
tool = message.tool_calls[0]
messages.append(message)

In [8]:
tool

ChatCompletionMessageToolCall(id='call_00_ta83vX1tilstuHNeeLGqf5x1', function=Function(arguments='{"location": "北京三里屯"}', name='get_weather'), type='function', index=0)

In [9]:
tool.id

'call_00_ta83vX1tilstuHNeeLGqf5x1'

In [10]:
function_str = tool.function
print(function_str)

Function(arguments='{"location": "北京三里屯"}', name='get_weather')


In [11]:
import json
import re

def extract_location_from_function_str(func_str):
    # 步骤1：用正则表达式提取 arguments 后的 JSON 字符串
    # 正则匹配规则：匹配 arguments='xxx' 中的 xxx（支持JSON中的双引号、逗号、冒号等符号）
    pattern = r"arguments='(\{.*?\})'"
    match = re.search(pattern, func_str, re.DOTALL)  # re.DOTALL 允许 . 匹配换行符（防止JSON换行）
    
    if not match:
        return "未找到 arguments 字段"
    
    # 步骤2：解析提取到的 JSON 字符串
    try:
        arguments_json = match.group(1)
        arguments_dict = json.loads(arguments_json)  # 转为字典
        # 步骤3：获取 location 字段（若不存在返回默认值）
        location = arguments_dict.get("location", "location 字段不存在")
        return location
    except json.JSONDecodeError:
        return "JSON 格式解析失败"
    except Exception as e:
        return f"提取失败：{str(e)}"

In [12]:
extracted_location_str = extract_location_from_function_str(str(function_str))
print(extracted_location_str)

北京三里屯


In [13]:
!pip install cryptography
!pip install --upgrade pyjwt

In [14]:
!python /Users/mac/xgshi/hefen-JWT-json-web-token/weather_project/get_weather_specific_city.py "$extracted_location_str" --config /Users/mac/xgshi/hefen-JWT-json-web-token/weather_project/config.ini --save --output "latest"

[信息] 成功从 /Users/mac/xgshi/hefen-JWT-json-web-token/weather_project 导入 HeWeatherAPI 核心模块
🌤️  和风天气命令行查询工具
[信息] HeWeatherAPI初始化成功，使用配置文件: /Users/mac/xgshi/hefen-JWT-json-web-token/weather_project/config.ini

🔍 正在搜索: 北京三里屯
[信息] JWT令牌生成成功，有效期 1800 秒
[信息] 找到 5 个匹配结果:
  1. 北京 (北京/北京市) - ID: 101010100
  2. 东城 (北京/北京市) - ID: 101011600
  3. 西城 (北京/北京市) - ID: 101011700
  4. 海淀 (北京/北京市) - ID: 101010200
  5. 朝阳 (北京/北京市) - ID: 101010300

🌤️  正在查询 北京 (北京/北京市) (ID: 101010100) 的天气...
[信息] 使用缓存的JWT令牌
[信息] 天气数据获取成功

🌍 实时天气 · 北京 (北京/北京市)
🌤️  天气状况: 晴
🌡️  温度: 10°C
🤔 体感温度: 5°C
💨 风向风力: 西北风 3级
💧 湿度: 17%
📊 气压: 1020百帕
👁️  能见度: 25公里
🕒 数据更新时间: 2025-12-07
[信息] 数据已保存至: latest.json
[信息] 结果已保存到: latest.json

💡 提示: 找到 5 个匹配结果，使用 --choose 参数手动选择
其他匹配结果:
  2. 东城 (北京/北京市) - ID: 101011600
  3. 西城 (北京/北京市) - ID: 101011700
  4. 海淀 (北京/北京市) - ID: 101010200
  ... 还有 1 个结果


In [ ]:
# result_json ="""{"code":"200","updateTime":"2025-12-04T16:42+08:00","fxLink":"https://www.qweather.com/weather/beijing-101010100.html","now":{"obsTime":"2025-12-04T16:40+08:00","temp":"5","feelsLike":"0","icon":"100","text":"晴","wind360":"0","windDir":"北风","windScale":"3","windSpeed":"14","humidity":"17","precip":"0.0","pressure":"1025","vis":"30","cloud":"0","dew":"-18"},"refer":{"sources":["QWeather"],"license":["QWeather Developers License"]}}%
# """

In [15]:
def read_latest_json(file_path="latest.json"):
    """
    读取 latest.json 文件，返回文件内容的字符串形式
    
    参数:
        file_path: json 文件路径（默认当前目录下的 latest.json）
                   若文件不在当前目录，需传入绝对路径（如 "/Users/mac/xxx/latest.json"）
    
    返回:
        str: 文件内容字符串（location_json）
    """
    try:
        # 以只读模式打开文件，指定 utf-8 编码避免中文乱码
        with open(file_path, "r", encoding="utf-8") as f:
            # 读取文件全部内容并赋值给 location_json（字符串类型）
            location_json = f.read()
        return location_json
    except FileNotFoundError:
        print(f"错误：未找到文件 {file_path}，请检查文件路径是否正确")
        return None
    except PermissionError:
        print(f"错误：没有读取 {file_path} 的权限")
        return None
    except Exception as e:
        print(f"读取文件失败：{str(e)}")
        return None

# 执行读取（默认读取当前目录下的 latest.json）
location_json = read_latest_json()

# 验证结果（可选，用于确认读取成功）
if location_json:
    print("文件读取成功！")
    print(f"location_json 变量类型：{type(location_json)}")  # 输出：<class 'str'>
    print("文件前 100 个字符预览：")
    print(location_json[:300])  # 打印前 300 字符，避免内容过长刷屏

文件读取成功！
location_json 变量类型：<class 'str'>
文件前 100 个字符预览：
{
  "code": "200",
  "updateTime": "2025-12-07T12:18+08:00",
  "fxLink": "https://www.qweather.com/weather/beijing-101010100.html",
  "now": {
    "obsTime": "2025-12-07T12:16+08:00",
    "temp": "10",
    "feelsLike": "5",
    "icon": "100",
    "text": "晴",
    "wind360": "315",
    "windDir": "西北


In [16]:
# 模拟 search_web 工具调用结果（直接返回24度）
# messages.append({"role": "tool", "tool_call_id": tool.id, "content": "1℃"})
messages.append({"role": "tool", "tool_call_id": tool.id, "content": location_json})
message = send_messages(messages)
print(f"Model>\t {message.content}")

Model>	 # 北京三里屯天气报告

## 📊 当前天气状况

**观测时间：** 2025年12月7日 12:16 (北京时间)

**天气状况：** ☀️ **晴**
- **温度：** 10°C
- **体感温度：** 5°C
- **湿度：** 17%
- **气压：** 1020 hPa

## 💨 风力信息
- **风向：** 西北风
- **风力等级：** 3级
- **风速：** 19 km/h

## 🌡️ 其他气象数据
- **降水量：** 0.0 mm
- **能见度：** 25 km
- **云量：** 0%
- **露点温度：** -16°C

## 📝 温馨提示
1. 天气晴朗，适合户外活动
2. 体感温度较低，建议适当增添衣物
3. 湿度较低，注意补水保湿
4. 能见度良好，适合出行

**数据更新时间：** 2025年12月7日 12:18  
**数据来源：** QWeather


In [17]:
print("summary: 第一次调用 LLM，LLM 根据情况决定要不要用 tool，给出 tool_call_id(例如， call_00_rgYsSOzshwdghkw6CSqRrwi0), \n然后 Agent 将 tool 执行的结果 append 到发给 LLM 的消息里面，再次调用 LLM，最终生成完整答案")

summary: 第一次调用 LLM，LLM 根据情况决定要不要用 tool，给出 tool_call_id(例如， call_00_rgYsSOzshwdghkw6CSqRrwi0), 
然后 Agent 将 tool 执行的结果 append 到发给 LLM 的消息里面，再次调用 LLM，最终生成完整答案


In [18]:
!cat latest.json

{
  "code": "200",
  "updateTime": "2025-12-07T12:18+08:00",
  "fxLink": "https://www.qweather.com/weather/beijing-101010100.html",
  "now": {
    "obsTime": "2025-12-07T12:16+08:00",
    "temp": "10",
    "feelsLike": "5",
    "icon": "100",
    "text": "晴",
    "wind360": "315",
    "windDir": "西北风",
    "windScale": "3",
    "windSpeed": "19",
    "humidity": "17",
    "precip": "0.0",
    "pressure": "1020",
    "vis": "25",
    "cloud": "0",
    "dew": "-16"
  },
  "refer": {
    "sources": [
      "QWeather"
    ],
    "license": [
      "QWeather Developers License"
    ]
  }
}

In [19]:
!pwd

/Users/mac/xgshi/code/jike-study/deepseek-quickstart/deepseek/rednote


 # 北京三里屯天气报告

## 📊 当前天气状况

**观测时间：** 2025年12月7日 12:16 (北京时间)

**天气状况：** ☀️ **晴**
- **温度：** 10°C
- **体感温度：** 5°C
- **湿度：** 17%
- **气压：** 1020 hPa

## 💨 风力信息
- **风向：** 西北风
- **风力等级：** 3级
- **风速：** 19 km/h

## 🌡️ 其他气象数据
- **降水量：** 0.0 mm
- **能见度：** 25 km
- **云量：** 0%
- **露点温度：** -16°C

## 📝 温馨提示
1. 天气晴朗，适合户外活动
2. 体感温度较低，建议适当增添衣物
3. 湿度较低，注意补水保湿
4. 能见度良好，适合出行

**数据更新时间：** 2025年12月7日 12:18  
**数据来源：** QWeather